In [2]:
# -----------------------
# Trace out the bifurcations for a given number of
# bounces where energy starts at -1/3. When a
# bifurcation is found, trace it out using a vector
# and varying p_phi
# -----------------------

# Import needed libraries
import magphyxp
import numpy as np
import sys
import queue

In [3]:
# -----------------------
# Variables deciding whether
# pphi is constant or ptheta
# is constant
VARY_PTHETA_PPHI = 0
VARY_PTHETA_ENERGY = 1
# -----------------------

def get_pphi_max(E, ptheta):
    x = (E + 1/3 - ptheta**2 / 2) / 5
    if x < 0:
        return -1
    return np.sqrt(x)

In [19]:
def create_state(mode_id, num_bounces, curr_state, min_two):
    return {
        'mode_id' : mode_id,
        'num_bounces' : num_bounces,
        'pr' : np.sqrt(abs(2*curr_state[2] + 2/3 - curr_state[0]**2 - 10*curr_state[1]**2)),
        'ptheta' : curr_state[0],
        'pphi' : curr_state[1],
        'energy' : curr_state[2],
        #'rocking_number' : int(min_two.rocking_number / 2),
        'ptheta_rocking_number' : min_two.ptheta_rocking_number,
        'pphi_rocking_number' : min_two.pphi_rocking_number,
        'in_phase' : min_two.rocking_in_phase,
        'period' : min_two.t,
        'theta_crossings' : min_two.theta_crossings,
        'phi_crossings' : min_two.phi_crossings,
        'beta_crossings' : min_two.beta_crossings,
    }    

def trace_mode(num_bounces, curr_state, mode_id, bifurcation_state):
    VECTOR_FACTOR = 1.1
    includeBif = False
    if includeBif:
        states = [create_state(mode_id, num_bounces, curr_state, bifurcation_state)]
    else:
        states = []
    # Used in determining if the energy is increasing within some margin
    epsilon = 1e-2
    #vector = np.array((0, V_STEP, 0)) # Setup vector
    vector = np.array((curr_state[0], curr_state[1], 0)) # Setup vector
    vector = vector*1e6
    #vector = np.array((V_STEP, V_STEP, 0)) # Setup vector
    new_bifurcation = True
    while (curr_state[2] < 0):
        next_state = curr_state + vector # Follow the vector to next state
        prev_state = next_state # Save state where we came from
        pphi_max = get_pphi_max(next_state[2], next_state[0])
        if (pphi_max < 0 or next_state[1] > pphi_max or next_state[2] > 0):
#             print('break 1.2')
            break
        # Calculate new min at new location
        min_two = magphyxp.calculate_min(next_state[0], next_state[1], num_bounces, next_state[2], 1e-7,
                                         VARY_PTHETA_ENERGY, 1e-7) 
        if min_two.f > 1e-9:
#             print('break 2: f={}'.format(min_two.f))
            break
        next_state = np.array((min_two.ptheta, min_two.pphi, min_two.energy)) # New min gives new state

        # Find new vector
        vector = next_state - curr_state

        curr_state = next_state # Change current state to the newest one found
        if (next_state[2] < (prev_state[2] - epsilon)): # Make sure energy is increasing within some tolerance
#             print('break 3')
            break
        # Write out ID, N, pr, ptheta, pphi, energy, rocking number, in phase, period
        #if new_bifurcation:
        #    new_bifurcation = False
        #    mode_id += 1
        state = create_state(mode_id, num_bounces, curr_state, min_two)
        states.append(state)
        # Slowly increase the step size of the vector
        vector[1] *= VECTOR_FACTOR
        vector[2] *= VECTOR_FACTOR
        # -------------------------------------------
        epsilon *= 0.9 # Slowly decrease tolerance of energy difference
    return states

# Globals

In [5]:
minimization_step_size = 1e-7
simulation_step_size = 1e-7

PTHETA_0 = 0
PPHI_0 = 1e-9
V_STEP = 1e-3

T_THRESHOLD = 350000


# Find a mode

In [11]:

def find_mode_step(num_bounces, E, verbose):
    global previous_E
    global bifurcation_id
    
    # Calculate minimum
    bifurcation_state = magphyxp.calculate_min(PTHETA_0, PPHI_0, num_bounces, E, minimization_step_size,
                                               VARY_PTHETA_ENERGY, simulation_step_size)

    if verbose:
        print('pre-candidate: {},{} '.format(bifurcation_state.ptheta_rocking_number,
                                             bifurcation_state.pphi_rocking_number,
                                            bifurcation_state.f))
    bifurcation_E = 1
    
    pphi_max = get_pphi_max(bifurcation_state.energy, bifurcation_state.ptheta)
    if (bifurcation_state.f < 1e-9 and bifurcation_state.energy > -1/3
        and bifurcation_state.energy - previous_E > E_step
        and V_STEP < pphi_max
#         and (bifurcation_state.rocking_number % num_bounces) > 0 # duplicate state
        and bifurcation_state.t > 0.0):
        # Get the current state of pphi, ptheta, and energy
        curr_state = np.array((bifurcation_state.ptheta, bifurcation_state.pphi, bifurcation_state.energy))
        bifurcation_E = bifurcation_state.energy
        
        if verbose:
            print('candidate: {},{} '.format(bifurcation_state.ptheta_rocking_number,
                                             bifurcation_state.pphi_rocking_number))

        states = trace_mode(num_bounces, curr_state, bifurcation_id, bifurcation_state)

        # We sometimes get phantom modes that have a low f value. This especially
        # occurs at low energy since the free magnet is moving very little. To ensure
        # that these don't get reported as true modes we go ahead and trace the mode
        # as far as it goes and then test if it reached a value of pphi above a threshold.
        # Keep the mode only if it reaches the threshold.
        if len(states) > 0 and states[-1]['pphi'] > 1e-3:
#             print(states[-1])
            ptheta_rocking_number = states[-1]['ptheta_rocking_number']
            pphi_rocking_number = states[-1]['pphi_rocking_number']
            print('{:.3f} ' .format(states[0]['energy']), end='', flush=True)
#             print('{},{} '.format(ptheta_rocking_number, pphi_rocking_number), end='', flush=True)
            if verbose:
                print()
            for state in states:
                f.write(
                    #'{} {} {:<.5f} {:<.5f} {:<.5f} {:<.5f} {} {} {:<.5f}\n'.format(
                    '{} {} {:<.5e} {:<.5e} {:<.5e} {:<.5e} {} {} {} {:<.5f} {} {} {}\n'.format(
                        state['mode_id'], state['num_bounces'], state['pr'],
                        state['ptheta'], state['pphi'],
                        state['energy'], state['ptheta_rocking_number'],  state['pphi_rocking_number'],
                        state['in_phase'], state['period'], state['theta_crossings'], state['phi_crossings'],
                        state['beta_crossings'])
                )

            f.flush()
            bifurcation_id += 1
            previous_E = bifurcation_state.energy

            if states[0]['period'] > T_THRESHOLD:
                return False
            
    return True


# Driver

In [20]:
import math

# fn = '/home/bojohnson/Desktop/traces.txt'
# if len(sys.argv) > 1:
#     fn = sys.argv[1]
fdir = './vis/states/'
bifurcation_id = 0

print('n  | E')
print('------------------------------------------------')

# TODO: put bounces to check
# All bounces used in the paper
bounces = [1]
# bounces = [1,2,3,4,5,6,7,9,11,13,15,19,23,27,31,39,47,55,63,79,111,157]

# Custom checks - don't remove these. If we need to rerun everything we'll need to rerun these.
# bounces = [1, 2, 9, 27, 27, 47, 47, 55, 55, 55, 63, 63, 79, 79, 95, 95, 111, 111, 111, 111, 127, 127, 127,
#            47, 55, 63, 79, 79, 111, 111, 111, 157, 157, 157, 157, 157, 157]

E_ranges = None

# Custom checks - don't remove these. If we need to rerun everything we'll need to rerun these.
# E_ranges = [[-0.0079,-0.00001], [-0.0079,-0.00001], [-0.1971,-0.1265], [-0.1868,-0.1551], [-0.1651,-0.1452],
#             [-0.2619,-0.2446], [-0.3020,-0.2869], [-0.2779,-0.2634], [-0.2418,-0.2268], [-0.3042,-0.2981],
#             [-0.2893,-0.2772], [-0.3106,-0.3058], [-0.3039,-0.2953], [-0.3185,-0.3152], [-0.3124,-0.3060],
#             [-0.3229,-0.3206], [-0.3177,-0.3129], [-0.3047,-0.3017], [-0.3299,-0.3272], [-0.3272,-0.3239],
#             [-0.3213,-0.3194], [-0.3110,-0.3086], [-0.3287,-0.3261], [-0.3333,-0.3312], [-0.3333,-0.3298],
#             [-0.3333,-0.3321], [-0.3303,-0.3286], [-0.3333,-0.3316], [-0.3313,-0.3299], [-0.3325,-0.3313],
#             [-0.3333,-0.3325], [-0.3253,-0.3241], [-0.3286,-0.3265], [-0.3303,-0.3286], [-0.3323,-0.3303],
#             [-0.3333,-0.3323], [-0.3303,-0.3286]]

for i, num_bounces in enumerate(bounces):
    print('{:<3d}| '.format(num_bounces), end='', flush=True)
    
    fn = fdir + 'states{:03d}.txt'.format(num_bounces)
    if E_ranges != None:
        lo = math.ceil(-E_ranges[i][0]*1000)
        hi = math.floor(-E_ranges[i][1]*1000)
        fn = fdir + 'states{:03d}_{}_{}.txt'.format(num_bounces, lo, hi)
        
    f = open(fn,'w') # File to write data to
    
    # For each bounce number start the search
    # with the given energy value E, the given V_STEP
    # which is the vector step size in tracing out
    # the bifurcation, and E_step which determines the
    # step for E
    
#     # ------------------------------------------------
#     # Binary search version
#     # m_max = 7
#     m_found = [False for i in range(m_max+1)]
#     bif_locs = [BifLoc(-1/3, 0), BifLoc(0, math.inf)]
#     find_mode_bsearch(num_bounces, -1/3, 0, m_found, bif_locs, m_max);
#     print()
    
    # ------------------------------------------------
    # Exhaustive search version
    E_start = -1/3 + 1e-3
    E_stop = 0
    E_step = 1e-3
    
    verbose = False
    if E_ranges != None:
        E_start = E_ranges[i][0]
        E_stop = E_ranges[i][1]
        num_steps = 1000
        # TODO CHANGE THIS BACK TO 1e-4!
#         E_step = 1e-4 #(E_stop - E_start) / num_steps
        E_step = 1e-5
    
    E = E_start

    previous_E = -1 # Placeholder for now

    keep_going = True
    while (keep_going and E <= E_stop):
        keep_going = find_mode_step(num_bounces, E, verbose);
        E = E + E_step
    print('wrote {}'.format(fn))

    f.close()
print('done')

n  | E
------------------------------------------------
1  | -0.065 -0.017 -0.007 -0.003 -0.000 wrote ./vis/states/states001.txt
done


# CSV convergence files

Produce two three-column files, the first giving the energies E and periods T of modes (m,1,1) for m = 3, 4, …, 100, and the second giving the E and T for modes (m,1,2) for m = 1, 2, …, 100.  The three-column format is

m, E, T

Plots made from these files will show the convergence of the numerics to the analytical results in the limit of large m. 

In [ ]:
def getRockingNumber(s):
#     if s['phase'] == 1:
#         # In-phase
#         return s['theta_crossings']
#     else:
#         # Out-of-phase
#         return s['pphi_rocking']
    if s.ptheta * s.pphi > 0:
        # In-phase
        return s.theta_crossings
    else:
        # Out-of-phase
        return s.pphi_rocking_number

def is_different_mode(a, b):
    if a == None or b == None:
        return True
    return (a.theta_crossings != b.theta_crossings or
            a.phi_crossings != b.phi_crossings or
            a.beta_crossings != b.beta_crossings or
            a.ptheta_rocking_number != b.ptheta_rocking_number or
            a.pphi_rocking_number != b.pphi_rocking_number)

def find_mode_rocking1(num_bounces, E, last_bifurcation, verbose):
    global previous_E
    global bifurcation_id
    
    # Calculate minimum
    bifurcation_state = magphyxp.calculate_min(PTHETA_0, PPHI_0, num_bounces, E, minimization_step_size,
                                               VARY_PTHETA_ENERGY, simulation_step_size)

    bifurcation_E = 1
    
    pphi_max = get_pphi_max(bifurcation_state.energy, bifurcation_state.ptheta)
#     if E > -0.008 and E < -0.0078:
#         print('{} {} {} {} {} {} {}'.format(bifurcation_state.f, bifurcation_state.energy, pphi_max, bifurcation_state.t, is_different_mode(bifurcation_state, last_bifurcation),
#                                            bifurcation_state.theta_crossings, bifurcation_state.pphi_rocking_number))
    if (bifurcation_state.f < 1e-9
        and bifurcation_state.energy > -1/3
#         and bifurcation_state.energy - previous_E > E_step
        and is_different_mode(bifurcation_state, last_bifurcation)
        and V_STEP < pphi_max
#         and (bifurcation_state.rocking_number % num_bounces) > 0 # duplicate state
        and bifurcation_state.t > 0.0
        and getRockingNumber(bifurcation_state) == 1):

        # Get the current state of pphi, ptheta, and energy
        curr_state = np.array((bifurcation_state.ptheta, bifurcation_state.pphi, bifurcation_state.energy))
        bifurcation_E = bifurcation_state.energy
        
        states = trace_mode(num_bounces, curr_state, bifurcation_id)

#         print('tracing {} {}'.format(bifurcation_state.energy, len(states)))
        # We sometimes get phantom modes that have a low f value. This especially
        # occurs at low energy since the free magnet is moving very little. To ensure
        # that these don't get reported as true modes we go ahead and trace the mode
        # as far as it goes and then test if it reached a value of pphi above a threshold.
        # Keep the mode only if it reaches the threshold.
        if len(states) > 0 and states[-1]['pphi'] > 1e-3:
            previous_E = bifurcation_state.energy
            return states[0], bifurcation_state
    return None, None

bifurcation_id = -1

def write_convergence_files():
    E_start = -1/3 + 1e-3
    E_stop = 0
    E_stop = -1/3# + 1e-3
    E_step = 1e-4
#     print('{}'.format((E_stop-E_start)/E_step))

    out_phase = False
    in_phase = False

    #---------------------
    # Loop through bounces looking for out of phase 
    #---------------------
    if out_phase:
        print('Looking for out of phase')
        conv_out_phase = open('./convergence_out_phase.csv', 'w')

        E_start = -0.05 # for num_bounces starting at 1
    #     E_start = -0.33 # for num_bounces starting at 22
    #     E_start = -0.3 # for num_bounces starting at 31
    #     E_start = -0.3322 # for num_bounces starting at 34
    #     E_start = -0.3325 # for num_bounces starting at 38
        for i in range(1, 101):
            if i >= 35:
                E_step = 1e-5
            if i >= 60:
                E_step = 1e-6
            if i >= 80:
                E_step = 1e-7

            num_bounces = i
            print('{:<3d}| '.format(num_bounces), end='', flush=True)

            verbose = False
            E = E_start

            previous_E = -1
            last_bifurcation = None

            found = False
    #         print('s{:.3f} '.format(E_step), end='', flush=True)
            while (not found and E > E_stop):
                state, bifurcation_state = find_mode_rocking1(num_bounces, E, last_bifurcation, verbose)
                if state != None and bifurcation_state.ptheta * bifurcation_state.pphi < 0:
                    print('{:.6f} '.format(state['energy']), end='', flush=True)
                    last_bifurcation = bifurcation_state
                    found = True
                    E_start = bifurcation_state.energy
                    conv_out_phase.write('{}, {}, {}\n'.format(
                        num_bounces, bifurcation_state.energy, bifurcation_state.t))

    #                 if bifurcation_state.ptheta * bifurcation_state.pphi > 0:
    #                     in_found = True
    #                     conv_in_phase.write('{}, {}, {}\n'.format(
    #                         num_bounces, bifurcation_state.energy, bifurcation_state.t))
    #                     E_stop = bifurcation_state.energy
    #                 else:
    #                     out_found = True
    #                     conv_out_phase.write('{}, {}, {}\n'.format(
    #                         num_bounces, bifurcation_state.energy, bifurcation_state.t))
                E = E - E_step
            print()
        conv_out_phase.close()

    #---------------------
    # Loop through bounces looking for in phase 
    #---------------------
    if in_phase:
        conv_in_phase = open('./convergence_in_phase.csv', 'w')
        print()
        print('Looking for in phase')
        E_step = 1e-3
        E_start = -0.0075 # for num_bounces = 3
    #     E_start = -0.26 # for num_bounces = 22
        for i in range(3, 101):
            if i >= 25:
                E_step = 1e-4
            if i >= 45:
                E_step = 1e-5
            if i >= 65:
                E_step = 1e-6
            if i >= 85:
                E_step = 1e-7
                
            num_bounces = i
            print('{:<3d}| '.format(num_bounces), end='', flush=True)

            verbose = False
            E = E_start

            previous_E = -1
            last_bifurcation = None

            keep_going = True
            found = False
    #         E_step = (E_stop - E_start)/333
    #         print('s{:.3f} '.format(E_step), end='', flush=True)
            while (not found and E > E_stop):
    #             print(E)
                state, bifurcation_state = find_mode_rocking1(num_bounces, E, last_bifurcation, verbose)
                if state != None and bifurcation_state.ptheta * bifurcation_state.pphi > 0:
                    print('{:.4f} '.format(state['energy']), end='', flush=True)
                    last_bifurcation = bifurcation_state
                    found = True
                    E_start = bifurcation_state.energy
                    conv_in_phase.write('{}, {}, {}\n'.format(
                        num_bounces, bifurcation_state.energy, bifurcation_state.t))

                E = E - E_step
            print()
        conv_in_phase.close()

    print('done')


write_convergence_files()